In [38]:
import numpy as np
import pandas as pd
import keras
import torch



In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
%run mrs.ipynb

# of rows of Song Data: 50683
# of unique songs: 50683
# of rows of User Listening Data: 806745
# of unique users: 25343
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50683 entries, 0 to 50682
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          50683 non-null  object 
 1   name              50683 non-null  object 
 2   artist            50683 non-null  object 
 3   spotify_id        50683 non-null  object 
 4   tags              49556 non-null  object 
 5   danceability      50683 non-null  float64
 6   energy            50683 non-null  float64
 7   loudness          50683 non-null  float64
 8   mode              50683 non-null  int64  
 9   speechiness       50683 non-null  float64
 10  acousticness      50683 non-null  float64
 11  instrumentalness  50683 non-null  float64
 12  liveness          50683 non-null  float64
 13  valence           50683 non-null  float64
 14  tempo        

In [36]:
#Note that the bert model and the mrs model need to run before this

x_test = np.load("misc/x_test.npy")
y_test = np.load("misc/y_test.npy")
model = keras.models.load_model("misc/mae_optimized_model.keras")
lyrics_embeddings_csv = pd.read_csv('misc/lyrics_embeddings.csv')
lyrics_embeddings_3d_csv = pd.read_csv('misc/lyrics_embeddings_3d.csv')
distance_frame = pd.read_csv('misc/distance_frame.csv')

lyrics_embeddings, lyrics_embeddings_3d = get_embeddings(lyrics_embeddings_csv, lyrics_embeddings_3d_csv)

In [37]:
POTENTIAL_N = 50

predicted = predict_sample(x_test[0], model)
recommendation = pipeline_helper(distance_frame, predicted, POTENTIAL_N, lyrics_embeddings)  
print(recommendation)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step


C:\Users\cam-m\AppData\Local\Temp\ipykernel_9888\323123518.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\cam-m\AppData\Local\Temp\ipykernel_9888\323123518.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



[9 7 8 5 6 4 3 0 1 2]
                       name  danceability  energy  loudness  speechiness  \
5          Somebody Told Me         0.508   0.979    -4.289       0.0847   
12           Feel Good Inc.         0.809   0.720    -7.776       0.1360   
13               Chop Suey!         0.420   0.925    -4.217       0.1200   
17               In the End         0.542   0.853    -6.407       0.0498   
21  Supermassive Black Hole         0.667   0.891    -5.126       0.0445   
29                     Numb         0.496   0.863    -4.153       0.0381   
34           Clint Eastwood         0.659   0.712    -7.602       0.1730   
44    Bitter Sweet Symphony         0.383   0.907    -5.409       0.0405   
47         Heart-Shaped Box         0.609   0.842    -5.861       0.0342   
48               By the Way         0.451   0.970    -4.938       0.1070   

    acousticness  instrumentalness  liveness  valence  
5       0.000087          0.000643    0.0641    0.704  
12      0.033700          0.0

### Spotify API Evaluation
Get recommendations from Spotify by passing in minimum and maximum feature values, and compare the songs to our recommended playlist.

In [ ]:
from spotify import SpotifyAPI
from dotenv import load_dotenv
# Get the input playlist
# Traverse along each song in playlist and preserve min max of feature values and 
# Store in list that gets passed to parameters for Spotify APIcommendations

input_playlist = og_songs
features = dict()
feature_names = ["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness", "valence"]

for feature in feature_names:
    features["min_" + feature] = input_playlist[feature].min()
    features["max_" + feature] = input_playlist[feature].max()

features["seed_genres"] = input_playlist['tags'].iloc[0].split(',')[:3]
features["seed_genres"] = ','.join(features["seed_genres"])
features

In [ ]:
env_path = os.path.join('misc', '.env')
load_dotenv(dotenv_path=env_path)
SPOTIFY_CREDS = [os.getenv('SPOTIFY_CLIENT_ID'), 
                os.getenv('SPOTIFY_CLIENT_SECRET')]

if not all(SPOTIFY_CREDS):
        print("Please set the SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET environment variables")
        exit(1)

spotify = SpotifyAPI(*SPOTIFY_CREDS)
SPOTIFY_CREDS

In [ ]:
recs_from_spotify = spotify.get_recommendations(features=features, limit=10)

In [ ]:
# SPOTIFY's PREDICTED RECOMMENDATIONS
recs_from_spotify = recs_from_spotify.loc[:, ['name', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                                'liveness', 'valence']]
recs_from_spotify

In [ ]:
def calculate_euclidean_distance(v1, v2):
    return np.linalg.norm(v1 - v2)

def calculate_score(pd1, pd2):
    if pd1.shape[1] != pd2.shape[1]:
        raise ValueError("Dataframes must have the same number of features.")
    
    for _, x in pd1.iterrows():
        fx = np.array(x[1:].values)
        dist = list()
        for _, y in pd2.iterrows():
            fy = np.array(y[1:].values)
            dist.append(calculate_euclidean_distance(fx, fy))
        print(np.mean(dist))